In [ ]:
# reference to : https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
# 사용할 라이브러리 선언
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

#for data preprocessing
from sklearn.decomposition import PCA

#for modeling
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split #training and testing data split
from keras.layers import Input, Dense
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
#filter warnings
import warnings
warnings.filterwarnings("ignore")

RANDOM_SEED = 42
random_dim = 121

In [ ]:
data=pd.read_csv("/content/drive/My Drive/merge_real_total.csv", header=None)  #정상 데이터 불러오기
data2 = pd.read_csv("/content/drive/My Drive/merge_paf_total.csv",header=None) #비정상 데이터 불러오기

In [ ]:
RANDOM_SEED=42
X_train,X_test=train_test_split(data,test_size=0.2,random_state=RANDOM_SEED) #train_test_split을 이용하여 전체 데이터에서 train용과 test용을 분리 (real data)
X_train = X_train.astype(float) / 255
X_train=X_train.dropna(axis=1) 
X_test = X_test.astype(float) / 255
X_test=X_test.dropna(axis=1)
input_dim = X_train.shape[1] 

In [ ]:
RANDOM_SEED = 42
X_train2,X_test2=train_test_split(data2,test_size=0.2,random_state=RANDOM_SEED) #train_test_split을 이용하여 전체 데이터에서 train용과 test용을 분리 (paf data)
X_train2 = X_train2.astype(float) / 255
X_train2=X_train2.dropna(axis=1) 
tmp=X_train2.to_numpy()
X_test2 = X_test2.astype(float) / 255
X_test2=X_test2.dropna(axis=1) #

In [ ]:
def get_generator():
        model = Sequential()
        model.add(layers.Dense(256, input_shape=(random_dim,)))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(512))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(1024))
        model.add(layers.LeakyReLU(alpha=0.2))
        model.add(layers.BatchNormalization())
        model.add(layers.Dense(121, activation='tanh'))
        model.compile(loss='binary_crossentropy', optimizer='Adam')
        return model


In [ ]:
def get_discriminator():
      model = Sequential()
      model.add(layers.Flatten(input_shape=(random_dim,)))
      model.add(layers.LeakyReLU(0.2))
      model.add(layers.Dense(512))
      model.add(layers.LeakyReLU(alpha=0.2))
      model.add(layers.Dense(256))
      model.add(layers.LeakyReLU(alpha=0.2))
      model.add(layers.Dense(1, activation='sigmoid'))
      model.compile(loss='binary_crossentropy', optimizer='Adam')
      return model

In [ ]:
def get_gan_network(discriminator, random_dim, generator):
    discriminator.trainable = False
    gan_input = Input(shape=(random_dim,))
    x = generator(gan_input)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

In [ ]:
epochs=100
batch_size=128
# train 데이터를 128 사이즈의 batch 로 나눕니다.
batch_count = X_train.shape[0] // batch_size

# 우리의 GAN 네트워크를 만듭니다.
generator = get_generator()
discriminator = get_discriminator()
gan = get_gan_network(discriminator, random_dim, generator)
k = 0
for e in range(1, epochs+1):
    for _ in tqdm(range(batch_count)):
        noise = np.random.normal(0, 1, size=[batch_size, random_dim]) #이걸 사용하는 이유는 0~1사이의 값을 가지는 난수를 발생시키는(생성자에 구분시키는 능력주려고)
        image_batch = X_train.to_numpy()[np.random.randint(0, X_train.to_numpy().shape[0], size=batch_size)]
        
        #print(image_batch)
        #l, image_batch = get_batch(k, batch_size)
        #print('changed:')
        #print(image_batch)
        #k = l
        
        generated_images = generator.predict(noise)
        X = np.concatenate([image_batch, generated_images]) #concatenate --> np 합쳐준다
        y_dis = np.zeros(2*batch_size)
        y_dis[:batch_size] = 0.9
        discriminator.trainable = True
        discriminator.fit(X, y_dis, validation_split=0.3, verbose=0)

        # Generator 학습
        noise = np.random.normal(0, 1, size=[batch_size, random_dim]) #generator는 가짜를 생성하도록 학습
        y_gen = np.ones(batch_size)
        gan.fit(noise, y_gen,shuffle=True,validation_split=0.3,verbose=0) #validation을 위하여 fit사용 validation은 입력데이터에서 6:4의 비율
model_json = generator.to_json() 
with open("/content/drive/My Drive/vanillagan_ecg.json", "w") as json_file: 
  json_file.write(model_json)
generator.save_weights("/content/drive/My Drive/vanillagan_ecg.h5") 
print("Saved model to disk")

In [ ]:
from tensorflow.compat.v2.keras.models import model_from_json


json_file = open("/content/drive/My Drive/vanillagan_ecg.json", "r") 
loaded_model_json = json_file.read() 
json_file.close()

loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("/content/drive/My Drive/vanillagan_ecg.h5") 
print("Loaded model from disk")

In [ ]:
generated_images2 = loaded_model.predict(X_test)  #real로 훈련된 생성자에 진짜 데이터를 입력으로 준다.
generated_images22 = loaded_model.predict(X_test2)  #real로 훈련된 생성자에 가짜 데이터를 입력으로 준다.

In [ ]:
import sklearn
x_sk=sklearn.metrics.pairwise.cosine_similarity(X_test.to_numpy(), generated_images2, dense_output=True) #real 재생성과 real test data 사이의 유사도 비교
# print(x_sk)
print(x_sk.mean())
x_sk=sklearn.metrics.pairwise.cosine_similarity(X_test.to_numpy(), generated_images22, dense_output=True) #paf 재생성과 paf test data 사이의 유사도 비교
# print(x_sk)
print(x_sk.mean())